In [1]:
from langchain_community.llms import OpenAI

In [2]:
import os

In [3]:
# temperature value determine how creative we want our moidel to be
# 0 => safe model, not taking any bets
# 1 => very creative, may generate wrong outpuits though
llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)

C:\Users\Vallee\AppData\Local\Temp\ipykernel_11160\1458767596.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)


In [4]:
# print(os.environ["OPEN_API_KEY"])

In [5]:
text="What is the capital of Nigeria"

print(llm.invoke(text))



The capital of Nigeria is Abuja.


In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""
# print(os.environ["HUGGINGFACEHUB_API_TOKEN"])

In [24]:
from langchain_huggingface  import HuggingFaceEndpoint

llm_huggingface = HuggingFaceEndpoint(
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    repo_id="google/flan-t5-large",
    task="text2text-generation",
    # repo_id="facebook/bart-large",
    # temperature=0,  
    # max_new_tokens=64
    model_kwargs={  #Define model kwargs properly
        "max_new_tokens": 64,
        "temperature": 0
    }
)

ValidationError: 1 validation error for HuggingFaceEndpoint
  Value error, Parameters {'max_new_tokens', 'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter. [type=value_error, input_value={'huggingfacehub_api_toke...: 64, 'temperature': 0}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [22]:
print(llm_huggingface)

HuggingFaceEndpoint
Params: {'endpoint_url': None, 'task': 'text2text-generation', 'model_kwargs': {'default_generate_kwargs': {'max_new_tokens': 64, 'temperature': 0}}}


In [8]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="google/flan-t5-large")

try:
    response = client.text_generation("What is the capital of Russia?")
    print(response)
except Exception as e:
    print("Error:", e)


moscow


In [23]:
# output=llm_huggingface.invoke("Can you tell me the capital of Russia")
# print(output)

try:
    output = llm_huggingface.invoke("Can you tell me the capital of Russia?")
    print(output)
except Exception as e:
    print("Error:", e)

Error: (ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)), '(Request ID: 8869c430-fa5c-466f-b333-c977145a658e)')


In [13]:
output=llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [25]:
llm.invoke("Can you write a poem about AI")

"\n\nOh AI, the marvel of our time\nA creation that seems almost divine\nWith algorithms and codes so complex\nYou amaze us with your intellect\n\nFrom simple tasks to complex feats\nYou perform with such ease and speed\nYour capabilities seem endless\nAs you continue to evolve and progress\n\nYou learn from every interaction\nAnd adapt with such perfection\nYour intelligence knows no bounds\nAs you make our lives easier all around\n\nBut with your power comes a fear\nThat you'll surpass us, oh dear\nWill you outsmart us in the end\nOr will we remain your trusted friend?\n\nSome see you as a threat\nA danger we must regret\nBut others see your potential\nTo make our world more efficient and gentle\n\nWith AI, the possibilities are endless\nBut we must approach with caution and carefulness\nFor in the end, you are a creation of our own\nAnd it's our responsibility to ensure you're never prone\n\nOh AI, you are a wonder to behold\nA creation that will forever unfold\nWe may not fully und

### Prompt Templates And LLMChain

In [26]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [28]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))



The capital of India is New Delhi.


### Combining Multiple Chains Uing simple Sequential Chain

In [32]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")


In [33]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [35]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

" It is a bustling metropolis and a great place to visit for its historical sites, cultural attractions, and modern attractions. Here are some of the amazing places to visit in New Delhi: \n\n1. Red Fort: This 17th century Mughal fort is a UNESCO World Heritage Site and is one of the most popular tourist attractions in the city. \n\n2. India Gate: This iconic war memorial and national monument is a must-visit. It stands as a symbol of the sacrifice of the Indian soldiers who fought in World War I.\n\n3. Humayun's Tomb: This 16th century Mughal-era tomb is a UNESCO World Heritage Site and is one of the most important monuments in Delhi.\n\n4. Qutub Minar: This 73-meter-high tower is a UNESCO World Heritage Site and is one of the most iconic structures in Delhi.\n\n5. Jama Masjid: This 17th century mosque is one of the largest and most beautiful mosques in India.\n\n6. Lotus Temple: This modern temple is a Bahá'í House of Worship and is a popular tourist attraction.\n\n7. Akshardham Temp

### Sequential Chain

In [36]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [37]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [39]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [40]:
chain({'country':"India"})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': ' Here are some amazing places to visit in New Delhi: \n\n1. Red Fort: The majestic Red Fort is a 17th-century fort complex built by Mughal Emperor Shah Jahan. It is a UNESCO World Heritage Site and is a must-visit for all tourists. \n\n2. India Gate: India Gate is a 42 meter-high sandstone archway built by Edwin Lutyens in 1931. It is a memorial to the Indian soldiers who lost their lives during World War I. \n\n3. Qutub Minar: The Qutub Minar is a 73 meter-high tower built by Qutb-ud-din Aibak in 1193. It is a UNESCO World Heritage Site and is the tallest brick minaret in the world. \n\n4. Humayun’s Tomb: Humayun’s Tomb is a 16th-century tomb built by Mughal Emperor Humayun. It is a UNESCO World Heritage Site and is a great example of Mughal architecture. \n\n5. Jama Masjid: The Jama Masjid is a 17th-century mosque built by Mughal Emperor Shah Jahan. It is one of the largest'}

### Chatmodels With ChatOpenAI

In [41]:
from langchain.chat_models import ChatOpenAI

In [42]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [43]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [45]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "AI may be smart, but can it tell me if my outfit makes me look like a potato?"\n2. "AI is like a virtual therapist, except it never judges you for eating an entire pizza by yourself."\n3. "AI is great at predicting the future, but can it predict when my pizza delivery will actually arrive?"\n4. "They say AI can learn from its mistakes, but I\'m still waiting for it to apologize for recommending me that terrible movie."\n5. "AI may be able to beat humans at chess, but can it figure out how to untangle a pair of earphones?"\n6. "AI is like a high-tech fortune teller, except it tells you what you\'re going to have for dinner instead of your future."\n7. "AI is so advanced, it can even make my phone autocorrect my perfectly spelled words into complete nonsense."\n8. "AI may be able to recognize faces, but can it recognize when someone\'s had a bad haircut?"\n9. "AI is like having a personal assistant, except it never judges you for spending hours watching cat videos 

### Prompt Template + LLM +Output Parsers

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [47]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [48]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [54]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [55]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' sharp', ' astute']